# Projeto MapBiomas: Pipeline de Desmatamento Municipal

## MVP: Engenharia de Dados

**Autor:** Laura Cardoso Gonzaga  
**Matrícula:** 4052025001151  
**Fonte dos dados:** [MapBiomas Brasil – Desmatamento Municípios (Coleção 10)](https://brasil.mapbiomas.org/estatisticas/)

## Objetivo

O projeto visa analisar padrões de desmatamento no Brasil com base nos dados do **MapBiomas** utilizando um pipeline de engenharia de dados em Databricks. O MapBiomas é uma rede global e multi-institucional, formada por universidades, ONGs e empresas de tecnologia, que monitora as transformações na cobertura e no uso da terra nos territórios e seus impactos. 

O foco está em transformar dados brutos a partir de uma arquitetura medalhão (camadas bronze, prata e ouro) em uma modelagem estrela confiável para análise. A partir disso, o projeto busca responder às seguintes perguntas:

- Quais biomas concentram as maiores áreas desmatadas ao longo do tempo?

- Quais municípios apresentaram os maiores volumes de desmatamento acumulado?

- Como evoluiu o desmatamento no Brasil ao longo dos anos?

- Que tipos de vegetação foram mais afetados?

- Qual a proporção de supressão de vegetação primária em relação à secundária?


## Importação de bibliotecas necessárias

In [0]:
%pip install xlrd==2.0.1 openpyxl

import pandas as pd
from pyspark.sql.functions import col

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Estrutura do Catálogo `mapbiomas`

In [0]:
'''
mapbiomas/
├── default/                 
│   └── data/                  ← Volume com dados originais
│       ├── mapbiomas_deforestation.xlsx
│       └── class_legend.csv
│       └── AR_BR_RG_UF_RGINT_RGI_MUN_2024.xls
│
├── bronze/                   ← Dados crus em formato Delta
│   └── mapbiomas_deforestation
│
├── silver/                   ← Dados transformados (stack, joins, cast, etc)
│   └── mapbiomas_deforestation
│
├── gold/                     ← Tabelas fato e dimensão
│   ├── fact_desmatamento
│   ├── dim_municipio
│   └── dim_classe_vegetacao
│
└── dim/                      ← Tabelas extras (legenda e dados externos)
    └── class_legend
    └── area_estados
'''

'\nmapbiomas/\n├── default/                 \n│   └── data/                  ← Volume com dados originais\n│       ├── mapbiomas_deforestation.xlsx\n│       └── class_legend.csv\n│       └── AR_BR_RG_UF_RGINT_RGI_MUN_2024.xls\n│\n├── bronze/                   ← Dados crus em formato Delta\n│   └── mapbiomas_deforestation\n│\n├── silver/                   ← Dados transformados (stack, joins, cast, etc)\n│   └── mapbiomas_deforestation\n│\n├── gold/                     ← Tabelas fato e dimensão\n│   ├── fact_desmatamento\n│   ├── dim_municipio\n│   └── dim_classe_vegetacao\n│\n└── dim/                      ← Tabelas extras (legenda e dados externos)\n    └── class_legend\n    └── area_estados\n'

In [0]:
%sql
USE CATALOG mapbiomas;

DROP SCHEMA IF EXISTS bronze CASCADE;
CREATE SCHEMA bronze;
DROP SCHEMA IF EXISTS silver CASCADE;
CREATE SCHEMA silver;
DROP SCHEMA IF EXISTS gold CASCADE;
CREATE SCHEMA gold;
DROP SCHEMA IF EXISTS dim CASCADE;
CREATE SCHEMA dim;

## Ingestão (Camada Bronze)

Nesta etapa, criamos a camada bronze com os arquivos raw, preservando os dados originais sem transformação. O objetivo é garantir reprodutibilidade e rastreabilidade, mantendo os dados brutos acessíveis e separados das demais etapas do pipeline.

#### Leitura dos dados brutos e escrita dos dados como tabelas Delta

In [0]:
df = pd.read_excel("/Volumes/mapbiomas/default/data/MAPBIOMAS_BRAZIL_DEFORESTATION_MUNICIPALITIES_COL.10_v2.xlsx",
                   sheet_name="DEFORESTATION") # tabela com os dados estruturados

df.head()

,country,biome,state,municipality,geocode_municipality,class,transition_name,class_level_0,class_level_1,class_level_2,class_level_3,class_level_4,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Brasil,Amazônia,Acre,Acrelândia,1200013,3,Supressão Veg. Primária,Natural,1. Forest,1.1. Forest Formation,1.1. Forest Formation,1.1. Forest Formation,3069.302624,2458.533382,1707.005027,1995.593391,1633.184204,2848.787854,3226.357614,3981.835159,2819.608795,7406.185622,3841.151243,4845.472404,6418.898221,4331.081910,5005.213898,8648.605263,6895.806627,5448.718021,6440.129732,5969.317489,3909.651690,3179.730386,3266.188063,3261.855308,2442.112236,1716.740162,1073.648401,2227.142798,1795.304982,1865.254408,1208.798112,1084.372794,1935.391300,2234.257690,1990.955063,2708.697398,2049.721612,1304.135691
1,Brasil,Amazônia,Acre,Acrelândia,1200013,3,Supressão Veg. Secundária,Natural,1. Forest,1.1. Forest Formation,1.1. Forest Formation,1.1. Forest Formation,0.000000,0.000000,8.368628,10.129967,4.757436,9.513449,55.143293,53.912194,63.688667,114.518828,42.020780,31.449999,109.942884,103.406794,115.214424,68.347912,60.427429,90.646494,118.734921,86.418173,38.228271,39.729683,104.913358,80.593423,60.426944,68.699107,38.665313,56.021198,51.972875,97.523997,77.890015,80.783762,165.689264,145.872219,229.833372,259.623443,367.193797,21.231191
2,Brasil,Amazônia,Acre,Acrelândia,1200013,6,Supressão Veg. Primária,Natural,1. Forest,1.4 Floodable Forest,1.4 Floodable Forest,1.4 Floodable Forest,2.818101,6.780473,10.214250,5.107059,0.792508,2.465625,3.962411,1.937604,0.000000,2.201680,2.554628,2.379490,0.792550,1.672828,0.792768,7.043595,5.989121,8.191717,23.244237,21.394423,2.553133,2.906325,2.465205,0.440777,9.331999,1.937527,0.352385,0.352137,0.000000,1.584510,0.616191,0.352112,0.088085,0.000000,18.837852,8.901130,3.964020,1.673282
3,Brasil,Amazônia,Acre,Acrelândia,1200013,6,Supressão Veg. Secundária,Natural,1. Forest,1.4 Floodable Forest,1.4 Floodable Forest,1.4 Floodable Forest,0.000000,0.000000,0.616376,0.176107,0.000000,0.000000,0.000000,0.968966,0.000000,0.000000,0.088092,0.352376,0.000000,0.792449,0.352141,0.088088,0.000000,0.176105,0.000000,0.000000,0.000000,1.408477,0.616631,0.176051,1.057054,0.088053,1.144914,0.088025,0.000000,0.000000,0.000000,0.440140,1.937809,0.792502,0.088027,0.528165,6.252777,0.440155
4,Brasil,Amazônia,Acre,Acrelândia,1200013,11,Supressão Veg. Primária,Natural,2. Non Forest Natural Formation,2.1. Wetland,2.1. Wetland,2.1. Wetland,7.575158,0.968577,4.403545,0.000000,0.704824,10.302555,3.258015,1.409513,0.000000,0.616533,1.496688,4.315147,1.232369,2.994916,0.176097,3.609126,0.088103,0.176122,0.000000,0.352214,1.585115,0.352143,0.352348,0.000000,1.056626,0.176109,0.000000,0.880610,0.000000,0.088054,0.000000,0.088092,0.000000,0.440487,0.000000,0.000000,0.176145,0.000000


In [0]:
df.isna().any()

country                 False
biome                   False
state                   False
municipality            False
geocode_municipality    False
class                   False
transition_name         False
class_level_0           False
class_level_1           False
class_level_2           False
class_level_3           False
class_level_4           False
1987                    False
1988                    False
1989                    False
1990                    False
1991                    False
1992                    False
1993                    False
1994                    False
1995                    False
1996                    False
1997                    False
1998                    False
1999                    False
2000                    False
2001                    False
2002                    False
2003                    False
2004                    False
2005                    False
2006                    False
2007                    False
2008      

In [0]:
df_bronze = spark.createDataFrame(df)

df_bronze.write.format("delta").mode("overwrite").saveAsTable("bronze.mapbiomas_deforestation")

In [0]:
df_bronze.printSchema()

root
 |-- country: string (nullable = true)
 |-- biome: string (nullable = true)
 |-- state: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- geocode_municipality: long (nullable = true)
 |-- class: long (nullable = true)
 |-- transition_name: string (nullable = true)
 |-- class_level_0: string (nullable = true)
 |-- class_level_1: string (nullable = true)
 |-- class_level_2: string (nullable = true)
 |-- class_level_3: string (nullable = true)
 |-- class_level_4: string (nullable = true)
 |-- 1987: double (nullable = true)
 |-- 1988: double (nullable = true)
 |-- 1989: double (nullable = true)
 |-- 1990: double (nullable = true)
 |-- 1991: double (nullable = true)
 |-- 1992: double (nullable = true)
 |-- 1993: double (nullable = true)
 |-- 1994: double (nullable = true)
 |-- 1995: double (nullable = true)
 |-- 1996: double (nullable = true)
 |-- 1997: double (nullable = true)
 |-- 1998: double (nullable = true)
 |-- 1999: double (nullable = true)
 |-- 2000: do

#### Legenda com tradução para o português

In [0]:
class_legend = pd.read_csv("/Volumes/mapbiomas/default/data/Codigos-da-legenda-colecao-10.csv",
                           delimiter= "\t")

class_legend.head(20)

,Class_ID,Level,Description,Descricao,Color
0,1,1,Forest,Floresta,#32a65e
1,3,2,Forest Formation,Formação Florestal,#1f8d49
2,4,2,Savanna Formation,Formação Savânica,#7dc975
3,5,2,Mangrove,Mangue,#04381d
4,6,2,Floodable Forest,Floresta Alagável,#026975
5,49,2,Wooded Sandbank Vegetation,Restinga Arbórea,#02d659
6,10,1,Herbaceous and Shrubby Vegetation,Vegetação Herbácea e Arbustiva,#ad975a
7,11,2,Wetland,Campo Alagado e Área Pantanosa,#519799
8,12,2,Grassland,Formação Campestre,#d6bc74
9,32,2,Hypersaline Tidal Flat,Apicum,#fc8114


In [0]:
df_legenda_spark = spark.createDataFrame(class_legend)

df_legenda_spark.write.format("delta").mode("overwrite").saveAsTable("dim.class_legend")

#### Dados externos IBGE: área dos estados

As áreas dos estados foram obtidas do [IBGE (2024)](https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/15761-areas-dos-municipios.html) e convertidas de km² para hectares.

In [0]:
df_area_estados = pd.read_excel(
    "/Volumes/mapbiomas/default/data/AR_BR_RG_UF_RGINT_RGI_MUN_2024.xls",
    sheet_name="AR_BR_UF_2024"
)

In [0]:
area_estados_spark = spark.createDataFrame(df_area_estados)

In [0]:
area_estados_min = (
    area_estados_spark
    .selectExpr(
        "NM_UF AS state",
        "CAST(AR_UF_2024 * 100 AS DOUBLE) AS area_uf_ha"     #  1 km² = 100 ha
    )
    .dropDuplicates(["state"])
    .filter(col("state").isNotNull())
)

area_estados_min.write.mode("overwrite").saveAsTable("dim.area_estados")

#### Metadados

In [0]:
metadados = pd.read_excel(
    "/Volumes/mapbiomas/default/data/MAPBIOMAS_BRAZIL_DEFORESTATION_MUNICIPALITIES_COL.10_v2.xlsx",
    sheet_name="METADADOS",
    header=2,
    index_col=0,
    usecols=range(1,4)
)

metadados.head(20)

,Descrição do campo,Field Description
Campo/Field,,
country,País,Country
biome,Bioma,Biome
state,Estado,State
municipality,Município,Municipality
geocode_municipality,Código do município IBGE,Municipality code (IBGE)
class,Código da legenda correspondente a classe do m...,Legend code for each deforestation module class
transition_name,Nome da classe do módulo de desmatamento,Name of the deforestation module class
class_level_1,Classe de cobertura e uso da terra no nível 1,Land cover and land use class on legend level 1
class_level_2,Classe de cobertura e uso da terra no nível 2,Land cover and land use class on legend level 2


## Transformação (Camada Silver)

A camada silver aplica as primeiras transformações estruturais como limpeza, padronização e enriquecimento com metadados. Aqui já obtemos uma tabela utilizável para análise, mas ainda não modelada para consulta analítica rápida.


In [0]:
%sql
DESCRIBE bronze.mapbiomas_deforestation;

col_name,data_type,comment
country,string,null
biome,string,null
state,string,null
municipality,string,null
geocode_municipality,bigint,null
class,bigint,null
transition_name,string,null
class_level_0,string,null
class_level_1,string,null
class_level_2,string,null


#### Conversão de colunas wide para long com `stack()`
Identificação automática das colunas de ano para transformar em um única coluna 'year'. Dessa forma, cada linha corresponde a uma observação, facilitando a agregação para análise.

In [0]:
year_cols = [c for c in df_bronze.columns if c.isdigit() and len(c) == 4]
year_cols = sorted(year_cols)

print(year_cols)

['1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


In [0]:
stack_expr = "stack({n}, {pairs}) as (year, area_ha)".format(
    n=len(year_cols),
    pairs=", ".join([f"'{y}', `{y}`" for y in year_cols])
)

#### Observando os dados
Agregação de categorias de classe conclui que 'class_level_2', 'class_level_3' e 'class_level_4' são reduntantes. Portanto, podemos manter apenas uma.

In [0]:
df_bronze.select("class", "class_level_0", "class_level_1", "class_level_2", "class_level_3", "class_level_4") \
    .distinct() \
    .orderBy("class") \
    .display()

class,class_level_0,class_level_1,class_level_2,class_level_3,class_level_4
3,Natural,1. Forest,1.1. Forest Formation,1.1. Forest Formation,1.1. Forest Formation
4,Natural,1. Forest,1.2. Savanna Formation,1.2. Savanna Formation,1.2. Savanna Formation
5,Natural,1. Forest,1.3. Mangrove,1.3. Mangrove,1.3. Mangrove
6,Natural,1. Forest,1.4 Floodable Forest,1.4 Floodable Forest,1.4 Floodable Forest
11,Natural,2. Non Forest Natural Formation,2.1. Wetland,2.1. Wetland,2.1. Wetland
12,Natural,2. Non Forest Natural Formation,2.2. Grassland,2.2. Grassland,2.2. Grassland
13,Natural,2. Non Forest Natural Formation,2.6. Other non Forest Formations,2.6. Other non Forest Formations,2.6. Other non Forest Formations
49,Natural,1. Forest,1.5. Wooded Sandbank Vegetation,1.5. Wooded Sandbank Vegetation,1.5. Wooded Sandbank Vegetation
50,Natural,2. Non Forest Natural Formation,2.4. Herbaceous Sandbank Vegetation,2.4. Herbaceous Sandbank Vegetation,2.4. Herbaceous Sandbank Vegetation


Brasil é o único país presente nos dados, logo essa também pode ser uma coluna descartável.

In [0]:
df_bronze.select("country").distinct().display()

country
Brasil


#### Criação de dataframe e tabela

In [0]:
dim_cols = [
    "geocode_municipality", # PK 
    "biome",
    "state",
    "municipality",
    "class",
    "transition_name",
    "class_level_1",
    "class_level_2"
]

In [0]:
df_silver = df_bronze.select(*dim_cols, *year_cols) \
    .selectExpr(*dim_cols, stack_expr) \
    .withColumn("year", col("year").cast("int"))

display(df_silver.limit(10))

geocode_municipality,biome,state,municipality,class,transition_name,class_level_1,class_level_2,year,area_ha
1200013,Amazônia,Acre,Acrelândia,3,Supressão Veg. Primária,1. Forest,1.1. Forest Formation,1987,3069.302624029574
1200013,Amazônia,Acre,Acrelândia,3,Supressão Veg. Secundária,1. Forest,1.1. Forest Formation,1987,0.0
1200013,Amazônia,Acre,Acrelândia,6,Supressão Veg. Primária,1. Forest,1.4 Floodable Forest,1987,2.81810064086914
1200013,Amazônia,Acre,Acrelândia,6,Supressão Veg. Secundária,1. Forest,1.4 Floodable Forest,1987,0.0
1200013,Amazônia,Acre,Acrelândia,11,Supressão Veg. Primária,2. Non Forest Natural Formation,2.1. Wetland,1987,7.575158142089847
1200013,Amazônia,Acre,Acrelândia,11,Supressão Veg. Secundária,2. Non Forest Natural Formation,2.1. Wetland,1987,0.0
1200013,Amazônia,Acre,Acrelândia,12,Supressão Veg. Primária,2. Non Forest Natural Formation,2.2. Grassland,1987,10.83564494018555
1200013,Amazônia,Acre,Acrelândia,12,Supressão Veg. Secundária,2. Non Forest Natural Formation,2.2. Grassland,1987,0.0
1200054,Amazônia,Acre,Assis Brasil,3,Supressão Veg. Primária,1. Forest,1.1. Forest Formation,1987,221.3526631408689
1200054,Amazônia,Acre,Assis Brasil,3,Supressão Veg. Secundária,1. Forest,1.1. Forest Formation,1987,0.0


In [0]:
df_silver.write.format("delta").mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("silver.mapbiomas_deforestation")
        

In [0]:
%sql
SELECT COUNT(*) as total,
       SUM(CASE WHEN area_ha IS NULL THEN 1 ELSE 0 END) AS nulos,
       SUM(CASE WHEN area_ha < 0 THEN 1 ELSE 0 END) AS negativos
FROM silver.mapbiomas_deforestation

total,nulos,negativos
1401782,0,0


#### Inclusão de nomes em português:
 Criação de colunas de classe florestal em português `class_level_1_pt` e `class_level_2_pt` com base em `class_legend`. A legenda não incluía a categoria "Outras Formações não Florestais" e foi incluída manualmente com CASE WHEN.



In [0]:
%sql
CREATE OR REPLACE TABLE silver.mapbiomas_deforestation AS
SELECT s.biome, s.state, s.municipality, s.geocode_municipality, s.transition_name, 
       s.class_level_1, 
       CASE WHEN s.class_level_1 = '1. Forest' THEN 'Floresta'
            WHEN s.class_level_1 = '2. Non Forest Natural Formation' THEN 'Formação não Florestal'
            ELSE NULL
            END AS class_level_1_pt,
       s.class_level_2, 
       CASE WHEN l.Descricao IS NOT NULL THEN l.Descricao
            WHEN s.class_level_2 = '2.6. Other non Forest Formations' THEN 'Outras Formações não Florestais'
            ELSE NULL
            END AS class_level_2_pt, 
       s.year, s.area_ha
FROM silver.mapbiomas_deforestation as s
LEFT JOIN dim.class_legend as l
ON s.class = l.Class_ID

num_affected_rows,num_inserted_rows


In [0]:
df_silver = spark.read.table("silver.mapbiomas_deforestation")

df_silver.groupBy("class_level_1", "class_level_1_pt", "class_level_2", "class_level_2_pt") \
    .count() \
    .orderBy("class_level_1") \
    .display()

class_level_1,class_level_1_pt,class_level_2,class_level_2_pt,count
1. Forest,Floresta,1.1. Forest Formation,Formação Florestal,471656
1. Forest,Floresta,1.3. Mangrove,Mangue,18316
1. Forest,Floresta,1.5. Wooded Sandbank Vegetation,Restinga Arbórea,17594
1. Forest,Floresta,1.2. Savanna Formation,Formação Savânica,288572
1. Forest,Floresta,1.4 Floodable Forest,Floresta Alagável,42712
2. Non Forest Natural Formation,Formação não Florestal,2.2. Grassland,Formação Campestre,271662
2. Non Forest Natural Formation,Formação não Florestal,2.1. Wetland,Campo Alagado e Área Pantanosa,275234
2. Non Forest Natural Formation,Formação não Florestal,2.4. Herbaceous Sandbank Vegetation,Restinga Herbácea,15960
2. Non Forest Natural Formation,Formação não Florestal,2.6. Other non Forest Formations,Outras Formações não Florestais,76


#### Adicionando metadados como comentários 

In [0]:
def set_comentario(table_name, metadados):
    table_cols = {row['col_name'] for row in spark.sql(f"DESCRIBE {table_name}").collect()}
    # Colunas não presentes no dataframe de metadados
    extras = {
        "class_level_1_pt": metadados.at["class_level_1", metadados.columns[0]] + " (em português)",
        "class_level_2_pt": metadados.at["class_level_2", metadados.columns[0]] + " (em português)",
        "year": "Ano",
        "area_ha": "Área em hectares"
    }
    comentarios = {campo: comentario for campo, comentario in zip(metadados.index, metadados.iloc[:,0])}
    comentarios.update(extras)
    for campo, comentario in comentarios.items():
        if campo in table_cols:
            spark.sql(f"ALTER TABLE {table_name} ALTER COLUMN `{campo}` COMMENT '{comentario}'")

set_comentario("silver.mapbiomas_deforestation", metadados)

In [0]:
%sql
DESCRIBE silver.mapbiomas_deforestation;

col_name,data_type,comment
biome,string,Bioma
state,string,Estado
municipality,string,Município
geocode_municipality,bigint,Código do município IBGE
transition_name,string,Nome da classe do módulo de desmatamento
class_level_1,string,Classe de cobertura e uso da terra no nível 1
class_level_1_pt,string,Classe de cobertura e uso da terra no nível 1 (em português)
class_level_2,string,Classe de cobertura e uso da terra no nível 2
class_level_2_pt,string,Classe de cobertura e uso da terra no nível 2 (em português)
year,int,Ano


## Modelagem Estrela (Camada Gold)

Na camada gold, reestruturamos os dados para análises otimizadas usando o modelo estrela (star schema), separando tabelas fato (valores quantitativos) e dimensão (atributos descritivos). A tabela fato guarda chaves estrangeiras que apontam para as dimensões. Essa estrutura facilita agregações, cruzamentos e visualizações com desempenho e clareza.

**Padrão de chaves**
- geocode_municipality → código IBGE
- state → nome completo (ex.: “Mato Grosso”)
- biome → nome conforme MapBiomas

![Modelo ER](https://github.com/lauragonzaga/MVP-Engenharia-de-Dados/blob/main/diagrama_ER.png?raw=true)

#### Tabelas Dimensão

In [0]:
%sql
CREATE OR REPLACE TABLE gold.dim_municipio AS
SELECT DISTINCT 
  geocode_municipality,
  municipality,
  state,
  biome
FROM silver.mapbiomas_deforestation

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE gold.dim_classe_vegetacao AS
SELECT DISTINCT
  hash(class_level_1, class_level_2) AS classe_id,
  class_level_1_pt,
  class_level_2_pt
FROM silver.mapbiomas_deforestation;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE gold.dim_transicao AS
SELECT DISTINCT
  hash(transition_name) AS transicao_id,
  transition_name
FROM silver.mapbiomas_deforestation
WHERE transition_name IS NOT NULL;

num_affected_rows,num_inserted_rows


#### Tabela Fato

In [0]:
%sql
CREATE OR REPLACE TABLE gold.fact_desmatamento AS
SELECT 
  geocode_municipality,
  hash(class_level_1, class_level_2) AS classe_id,
  hash(transition_name) AS transicao_id,
  year,
  area_ha
FROM silver.mapbiomas_deforestation;

num_affected_rows,num_inserted_rows


#### Checagem
Garantindo integridade conferindo se há chaves órfãs.

In [0]:
%sql
SELECT
  (SELECT COUNT(*) 
  FROM gold.fact_desmatamento f 
  LEFT ANTI JOIN gold.dim_municipio d 
  ON f.geocode_municipality = d.geocode_municipality) AS missing_municipios,
  (SELECT COUNT(*) 
  FROM gold.fact_desmatamento f 
  LEFT ANTI JOIN gold.dim_transicao d 
  ON f.transicao_id = d.transicao_id) AS missing_transicao,
  (SELECT COUNT(*) 
  FROM gold.fact_desmatamento f 
  LEFT ANTI JOIN gold.dim_classe_vegetacao d 
  ON f.classe_id = d.classe_id) AS missing_classes

missing_municipios,missing_transicao,missing_classes
0,0,0


## Exploração

Nesta etapa, utilizamos as tabelas da camada _gold_ para produzir consultas SQL que respondam às perguntas de negócio definidas no início. Essas consultas foram utitilizadas para construir um [dashboard](https://dbc-b08da042-4e74.cloud.databricks.com/dashboardsv3/01f0d08344e91b4791a28f4d268be1dd/published?o=1041144495999733) como produto final da análise.


In [0]:
%sql
-- Cobertura temporal
SELECT MIN(year) AS ano_inicial, MAX(year) AS ano_final
FROM gold.fact_desmatamento;

ano_inicial,ano_final
1987,2024


In [0]:
%sql
-- Total de área nativa e acumulada desmatada no Brasil
SELECT ROUND(SUM(CASE WHEN transition_name = 'Supressão Veg. Primária' 
                 THEN area_ha ELSE 0 END),2) as area_nativa_desmatada,
      ROUND(SUM(area_ha),2) as area_desmatada_acumulada
FROM gold.fact_desmatamento g
JOIN gold.dim_transicao m
  ON g.transicao_id = m.transicao_id


area_nativa_desmatada,area_desmatada_acumulada
1.4486848425E8,1.7622854824E8


In [0]:
%sql
-- Série temporal histórica nacional 

SELECT year, ROUND(SUM(area_ha), 2) AS area_total_ha
FROM gold.fact_desmatamento
GROUP BY year
ORDER BY year;

year,area_total_ha
1987,5647033.74
1988,5580900.85
1989,5158285.02
1990,4848121.02
1991,4884244.17
1992,5760493.03
1993,5222493.16
1994,5459509.55
1995,6224311.54
1996,5611487.63


In [0]:
%sql
-- Evolução do desmatamento acumulado por bioma

SELECT biome, year, ROUND(SUM(area_ha),2) AS area_total_ha
FROM gold.fact_desmatamento g
JOIN gold.dim_municipio m
ON g.geocode_municipality = m.geocode_municipality
GROUP BY year, biome
ORDER BY year, biome

biome,year,area_total_ha
Amazônia,1987,1975012.69
Caatinga,1987,696272.78
Cerrado,1987,2999296.45
Mata Atlântica,1987,1150831.0
Pampa,1987,209591.64
Pantanal,1987,193954.84
Amazônia,1988,1759695.24
Caatinga,1988,740738.18
Cerrado,1988,2856497.19
Mata Atlântica,1988,1203512.59


In [0]:
%sql
-- Top 10 estados com maior desmatamento acumulado
SELECT 
  m.state, 
  ROUND(SUM(g.area_ha),2) AS area_desmatada_acumulada
FROM gold.fact_desmatamento g
JOIN gold.dim_municipio m ON g.geocode_municipality = m.geocode_municipality
JOIN gold.dim_transicao t ON g.transicao_id = t.transicao_id
GROUP BY m.state
ORDER BY area_desmatada_acumulada DESC
LIMIT 10

state,area_desmatada_acumulada
Mato Grosso,4.523567371E7
Pará,2.970735232E7
Bahia,2.134230516E7
Minas Gerais,1.587357796E7
Maranhão,1.454250159E7
Rio Grande do Sul,1.302828602E7
Tocantins,1.299893727E7
Mato Grosso do Sul,1.184878251E7
Rondônia,1.03206583E7
Goiás,9502920.3


In [0]:
%sql
-- Top 10 estados com maior percentual de área nativa desmatada

WITH area_nativa_uf AS (
  SELECT 
    m.state,
    ROUND(SUM(CASE 
      WHEN t.transition_name = 'Supressão Veg. Primária' 
      THEN g.area_ha ELSE 0 END), 2) AS area_nativa_desmatada
  FROM gold.fact_desmatamento g
  JOIN gold.dim_transicao t 
    ON g.transicao_id = t.transicao_id
  JOIN gold.dim_municipio m 
    ON g.geocode_municipality = m.geocode_municipality
  GROUP BY m.state
)

SELECT 
  a.state,
  a.area_nativa_desmatada,
  ROUND(a.area_nativa_desmatada / e.area_uf_ha * 100, 2) AS perc_desmatado
FROM area_nativa_uf a
JOIN dim.area_estados e 
  ON a.state = e.state
ORDER BY perc_desmatado DESC
LIMIT 10


state,area_nativa_desmatada,perc_desmatado
Mato Grosso,4.055999277E7,44.91
Rondônia,9665801.63,40.65
Tocantins,1.072780893E7,38.67
Maranhão,1.185536503E7,35.96
Rio Grande do Sul,8667168.32,30.77
Bahia,1.665009897E7,29.48
Mato Grosso do Sul,1.040619055E7,29.14
Sergipe,588830.72,26.84
Ceará,3860336.07,25.93
Piauí,6422651.4,25.51


In [0]:
%sql
-- Top 10 municípios com maior desmatamento acumulado

SELECT CONCAT(municipality, ', ', state) AS municipio, 
       ROUND(SUM(area_ha), 2) AS area_total_ha
FROM gold.fact_desmatamento g
JOIN gold.dim_municipio m
ON g.geocode_municipality = m.geocode_municipality
GROUP BY m.municipality, m.state
ORDER BY area_total_ha DESC
LIMIT 10

municipio,area_total_ha
"São Félix do Xingu, Pará",2138865.15
"Paranatinga, Mato Grosso",1802867.29
"Cáceres, Mato Grosso",1774184.43
"Santa Maria das Barreiras, Pará",1311207.8
"Santana do Araguaia, Pará",1296521.18
"Brasnorte, Mato Grosso",1229349.12
"Porto Velho, Rondônia",1224879.52
"Altamira, Pará",1219260.1
"Nova Ubiratã, Mato Grosso",1162296.0
"Corumbá, Mato Grosso do Sul",1066775.57


In [0]:
%sql
-- Tipo de vegetação mais desmatado

SELECT class_level_1_pt, c.class_level_2_pt, ROUND(SUM(f.area_ha), 2) AS area_total_ha
FROM gold.fact_desmatamento f
JOIN gold.dim_classe_vegetacao c ON f.classe_id = c.classe_id
GROUP BY class_level_1_pt, c.class_level_2_pt
ORDER BY area_total_ha DESC;

class_level_1_pt,class_level_2_pt,area_total_ha
Floresta,Formação Florestal,8.684739797E7
Floresta,Formação Savânica,6.881205983E7
Formação não Florestal,Formação Campestre,1.334650823E7
Floresta,Floresta Alagável,3640076.95
Formação não Florestal,Campo Alagado e Área Pantanosa,3149397.26
Floresta,Restinga Arbórea,229834.8
Formação não Florestal,Restinga Herbácea,198709.06
Floresta,Mangue,4562.47
Formação não Florestal,Outras Formações não Florestais,1.68


In [0]:
%sql
-- Supressão primária vs. secundária

SELECT t.transition_name, ROUND(SUM(f.area_ha), 2) AS area_total_ha
FROM gold.fact_desmatamento f
JOIN gold.dim_transicao t ON f.transicao_id = t.transicao_id
GROUP BY t.transition_name
ORDER BY area_total_ha DESC;

transition_name,area_total_ha
Supressão Veg. Primária,1.4486848425E8
Supressão Veg. Secundária,3.136006399E7


In [0]:
%sql
-- Evolução supressão primária vs. secundária
SELECT transition_name, year, ROUND(SUM(area_ha),2) AS area_total_ha
FROM gold.fact_desmatamento g
JOIN gold.dim_transicao m
ON g.transicao_id = m.transicao_id
GROUP BY year, transition_name
ORDER BY year, transition_name

transition_name,year,area_total_ha
Supressão Veg. Primária,1987,5647033.74
Supressão Veg. Secundária,1987,0.0
Supressão Veg. Primária,1988,5580900.85
Supressão Veg. Secundária,1988,0.0
Supressão Veg. Primária,1989,5124286.32
Supressão Veg. Secundária,1989,33998.7
Supressão Veg. Primária,1990,4753595.85
Supressão Veg. Secundária,1990,94525.17
Supressão Veg. Primária,1991,4691199.96
Supressão Veg. Secundária,1991,193044.21


## Conclusões

- Como era de se esperar, o bioma com maior volume de desmatamento acumulado é a Amazônia. No entanto, em 2023 e 2024, seu desmatamento caiu drasticamente e o Cerrado a ultrapassou como bioma mais desmatado do ano.
- O desmatamento da Caatinga tem crescido bastante desde 2020, também superando a Amazônia em 2024.
- A vegetação mais desmatada é a Formação Florestal.
- O estado do Mato Grosso apresenta o maior volume de desmatamento, tanto em volume absoluto de vegetação nativa (primária) desmatada, quanto no total acumulado. É também o estado com a maior porcentagem de sua vegetação nativa desmatada (cerca de 45%), seguido de Rondônia, Tocantins e Maranhão.
- A supressão de vegetação primária representa a maior parcela do desmatamento anual, mas a secundária tem crescido continuamente nos últimos anos.

## Próximos passos

- Incorporar dados de políticas públicas e eventos nacionais (ex: criação de áreas protegidas, campanhas de fiscalização, mandatos presidenciais, etc.) para avaliar correlações com o desmatamento.
- Cruzar com dados da COMEX (comércio exterior) para investigar possível vínculo entre desmatamento e exportação de commodities
- Estender a análise com outras bases de dados do MapBiomas, como Cicatrizes de Fogo, Mineração e Agricultura.